In [163]:
from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.naive_bayes import GaussianNB

### Loading the cleaned dataset

In [164]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

In [165]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
columns_to_exclude = ['sex', 'ability to speak english', 'gave birth this year']
# Encoding the features and target, and excluding some columns
X_encoded, y_encoded = encode_all_features(X, y, [])
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

### Model

In [166]:
# Naive Bayes model (Gaussian)
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Predictions
nb_preds = nb_model.predict(X_test)
# Accuracy evaluation
nb_accuracy = accuracy_score(y_test, nb_preds)

print(classification_report(y_test, nb_preds))
print("Naive Bayes Accuracy:", nb_accuracy)

              precision    recall  f1-score   support

           0       0.82      0.73      0.78      1175
           1       0.58      0.71      0.64       625

    accuracy                           0.72      1800
   macro avg       0.70      0.72      0.71      1800
weighted avg       0.74      0.72      0.73      1800

Naive Bayes Accuracy: 0.7227777777777777


In [167]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import GaussianNB, CategoricalNB

class CombinedNB:
    """
    Combined Naive Bayes model for mixed data types (categorical and numerical)
    This looks a bit like a sklearn classifier class, but it's not (it doesn't inherit from anything). It's just a simple class that implements the fit and predict methods only for two combined Naive Bayes models.
    """
    def __init__(self, num_feat: List[str], cat_model: CategoricalNB = CategoricalNB(), num_model: GaussianNB = GaussianNB()):
        self.cat_model: CategoricalNB = cat_model
        self.num_model: GaussianNB = num_model
        self.num_feat: List[str] = num_feat # numerical features

    def fit(self, X_train, y_train):
        """
        Fit the combined model to the training data. We fit the categorical model to the categorical features and the numerical model to the numerical features.
        :param X_train: training dataset with mixed features
        :param y_train: training dataset with target variable
        :return: 
        """
        # Splitting data into categorical and numerical features
        X_train_cat = X_train.drop(columns=self.num_feat)
        X_train_num = X_train[self.num_feat]

        self.cat_model.fit(X_train_cat, y_train)
        self.num_model.fit(X_train_num, y_train)

    def predict(self, X_test):
        """
        Predict the target variable for the test data.
        We use the formula: 
        P(Class | C1, ..., Cn, N1, ..., Nm) ~= Mult{1, n}(P(Ci | Class)) * Mult{1, m}(P(Nj | Class)) * P(Class)
        where Ci are the categorical features, Nj are the numerical features, and Class is the target variable. 
        Concerning the scores, we get the following:
        Cat * Num * Prior = (Score_cat * Score_num) / Prior
        :param X_test: test dataset with mixed features
        :return: predicted target variable
        """
        cat_probs = self.cat_model.predict_proba(X_test.drop(columns=self.num_feat))
        num_probs = self.num_model.predict_proba(X_test[self.num_feat])
        combined_probs = (cat_probs * num_probs) / self.num_model.class_prior_
        combined_probs /= combined_probs.sum(axis=1, keepdims=True)
        return np.argmax(combined_probs, axis=1)

In [168]:
def test_combined_nb_model():
    data_ = {
    'Refund': ['Yes', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No', 'No', 'No'],
    'Marital Status': ['Single', 'Married', 'Single', 'Married', 'Divorced', 'Married', 'Divorced', 'Single', 'Married', 'Single', 'Divorced'],
    'Taxable Income': [125, 100, 70, 120, 95, 60, 220, 85, 75, 90, 120],
    'Evade': ['No', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'Yes', 'No']
    }
    
    df = pd.DataFrame(data_)
    
    # Encoding the features and target
    le_ = LabelEncoder()
    for col in df.columns:
        if col not in ['Taxable Income']:
            enc = le_.fit_transform(df[col])
            df[col] = enc
            # print encoding mapping
            print(f"Mapping for {col}:")
            for original, encoded in zip(le_.classes_, range(len(le_.classes_))):
                print(f"{original} -> {encoded}")
    
    # Separate features (X) and target variable (y)
    X_ = df.iloc[:, :-1]  # All columns except the last one
    y_ = df.iloc[:, -1]   # Last column (Evade)
    
    # Splitting data into training and test sets
    X_train_ = X_.iloc[:-1]  # All rows except the last one
    X_test_ = X_.iloc[-1:]   # Last row
    y_train_ = y_.iloc[:-1]  # All rows except the last one
    y_test_ = y_.iloc[-1:]   # Last row
    
    comb_nb = CombinedNB(num_feat=['Taxable Income'])
    comb_nb.fit(X_train_, y_train_)
    comb_nb_preds = comb_nb.predict(X_test_)
    
    assert (comb_nb.cat_model.category_count_[0][0] == [4., 3.]).all()
    assert (comb_nb.cat_model.category_count_[0][1] == [3., 0.]).all()
    assert (comb_nb.cat_model.category_count_[1][0] == [1., 4., 2.]).all()
    assert (comb_nb.cat_model.category_count_[1][1] == [1., 0., 2.]).all()
    assert (comb_nb.num_model.class_count_ == np.array([7., 3.])).all()
    assert (comb_nb.num_model.class_prior_ == np.array([0.7, 0.3])).all()
    assert (comb_nb.num_model.theta_ == np.array([[110.], [ 90.]])).all()
    assert (np.round(comb_nb.num_model.var_) == np.round(np.array([[2550.00000187], [16.66666854]]))).all()
    assert comb_nb_preds == 0
    
    # convert log probabilities to probabilities
    feature_probs_0 = np.exp(comb_nb.cat_model.feature_log_prob_[0])
    feature_probs_1 = np.exp(comb_nb.cat_model.feature_log_prob_[1])
    
    print(feature_probs_0)
    print(feature_probs_1)

In [169]:
test_combined_nb_model()

Mapping for Refund:
No -> 0
Yes -> 1
Mapping for Marital Status:
Divorced -> 0
Married -> 1
Single -> 2
Mapping for Evade:
No -> 0
Yes -> 1
[[0.55555556 0.44444444]
 [0.8        0.2       ]]
[[0.2        0.5        0.3       ]
 [0.33333333 0.16666667 0.5       ]]


In [170]:
X_encoded = X.copy()
# encode all categorical features using label encoding
le = LabelEncoder()
for col in X_encoded.columns:
    if col not in ['education', 'ability to speak english', 'age', 'workinghours']:
        X_encoded[col] = le.fit_transform(X[col])
                
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

comb_nb = CombinedNB(num_feat=['age', 'workinghours'])
comb_nb.fit(X_train, y_train)
comb_nb_preds = comb_nb.predict(X_test)
comb_nb_accuracy = accuracy_score(y_test, comb_nb_preds)

print("Combined Naive Bayes Accuracy:", comb_nb_accuracy)
print(classification_report(y_test, comb_nb_preds))

Combined Naive Bayes Accuracy: 0.7738888888888888
              precision    recall  f1-score   support

           0       0.84      0.81      0.82      1175
           1       0.66      0.72      0.69       625

    accuracy                           0.77      1800
   macro avg       0.75      0.76      0.76      1800
weighted avg       0.78      0.77      0.78      1800


In [171]:
X_encoded = X.copy()
X_encoded = X_encoded.drop(columns=['age', 'workinghours'])
# X_encoded['age'] = pd.cut(X['age'], bins=[0,28,38,49,65,93], labels=['(17-28]', '(28-38]', '(38-49]', '(49-65]', '(65-93]'])
X_encoded['workinghours'] = pd.cut(X['workinghours'], bins=[0, 30, 40, 99], labels=['Part-time', 'Full-time', 'Overtime'])

# encode all categorical features using label encoding
le = LabelEncoder()
for col in X_encoded.columns:
    if col not in ['education', 'ability to speak english']:
        X_encoded[col] = le.fit_transform(X[col])

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

nb_cat = CategoricalNB()
nb_cat.fit(X_train, y_train)

# Predictions
nb_preds = nb_cat.predict(X_test)
# Accuracy evaluation
nb_accuracy = accuracy_score(y_test, nb_preds)

print(classification_report(y_test, nb_preds))
print("Naive Bayes Accuracy:", nb_accuracy)

              precision    recall  f1-score   support

           0       0.83      0.84      0.83      1175
           1       0.69      0.67      0.68       625

    accuracy                           0.78      1800
   macro avg       0.76      0.76      0.76      1800
weighted avg       0.78      0.78      0.78      1800

Naive Bayes Accuracy: 0.7811111111111111


In [172]:
# save model
save_model(nb_model, '../output/saved_models/naive_bayes_model.joblib')